In [1]:
import cv #Opencv
from PIL import Image #Image from PIL
import glob
import os
import numpy

In [2]:
def DetectFace(image, faceCascade, returnImage=False):
    # This function takes a grey scale cv image and finds
    # the patterns defined in the haarcascade function
    # modified from: http://www.lucaamore.com/?p=638

    #variables    
    min_size = (20,20)
    haar_scale = 1.1
    min_neighbors = 3
    haar_flags = 0

    # Equalize the histogram
    cv.EqualizeHist(image, image)

    # Detect the faces
    faces = cv.HaarDetectObjects(
            image, faceCascade, cv.CreateMemStorage(0),
            haar_scale, min_neighbors, haar_flags, min_size
        )

    # If faces are found
    if faces and returnImage:
        for ((x, y, w, h), n) in faces:
            # Convert bounding box to two CvPoints
            pt1 = (int(x), int(y))
            pt2 = (int(x + w), int(y + h))
            cv.Rectangle(image, pt1, pt2, cv.RGB(255, 0, 0), 5, 8, 0)

    if returnImage:
        return image
    else:
        return faces

def pil2cvGrey(pil_im):
    # Convert a PIL image to a greyscale cv image
    # from: http://pythonpath.wordpress.com/2012/05/08/pil-to-opencv-image/
    pil_im = pil_im.convert('L')
    cv_im = cv.CreateImageHeader(pil_im.size, cv.IPL_DEPTH_8U, 1)
    cv.SetData(cv_im, pil_im.tobytes(), pil_im.size[0]  )
    return cv_im

def cv2pil(cv_im):
    # Convert the cv image to a PIL image
    return Image.fromstring("L", cv.GetSize(cv_im), cv_im.tostring())

def imgCrop(image, cropBox, boxScale=1):
    # Crop a PIL image with the provided box [x(left), y(upper), w(width), h(height)]

    # Calculate scale factors
    xDelta=max(cropBox[2]*(boxScale-1),0)
    yDelta=max(cropBox[3]*(boxScale-1),0)

    # Convert cv box to PIL box [left, upper, right, lower]
    PIL_box=[cropBox[0]-xDelta, cropBox[1]-yDelta, cropBox[0]+cropBox[2]+xDelta, cropBox[1]+cropBox[3]+yDelta]

    return image.crop(PIL_box)
count=0
def faceCrop(imagePattern,boxScale=1):
    # Select one of the haarcascade files:
    #   haarcascade_frontalface_alt.xml  <-- Best one?
    #   haarcascade_frontalface_alt2.xml
    #   haarcascade_frontalface_alt_tree.xml
    #   haarcascade_frontalface_default.xml
    #   haarcascade_profileface.xml
    #path = '/usr/local/Cellar/opencv/2.4.12_2/lib/python2.7/site-packages/haarcascade_frontalface_alt.xml'
    #faceCascade = numpy.asarray(cv.Load(path, cv.CreateMemStorage(), 'faceCascade'))

    #faceCascade = cv.Load('/Users/Singsing/Downloads/haarcascade_frontalface_default.xml')
    faceCascade = cv.Load('/Users/Singsing/Downloads/frontalFace10/haarcascade_frontalface_alt2.xml')

    imgList=glob.glob(imagePattern)
    if len(imgList)<=0:
        print 'No Images Found'
        return

    for img in imgList:
        pil_im=Image.open(img)
        cv_im=pil2cvGrey(pil_im)
        faces=DetectFace(cv_im,faceCascade)
        if faces:
            n=1
            for face in faces:
                croppedImage=imgCrop(pil_im, face[0],boxScale=boxScale)
                fname,ext=os.path.splitext(img)
                croppedImage.save(fname+'_crop'+str(n)+ext)
                n+=1
        else:
            print 'No faces found:', img
        
            
def test(imageFilePath):
    pil_im=Image.open(imageFilePath)
    cv_im=pil2cvGrey(pil_im)
    # Select one of the haarcascade files:
    #   haarcascade_frontalface_alt.xml  <-- Best one?
    #   haarcascade_frontalface_alt2.xml
    #   haarcascade_frontalface_alt_tree.xml
    #   haarcascade_frontalface_default.xml
    #   haarcascade_profileface.xml
    faceCascade = cv.Load('haarcascade_frontalface_alt.xml')
    face_im=DetectFace(cv_im,faceCascade, returnImage=True)
    img=cv2pil(face_im)
    img.show()
    img.save('test.png')


In [3]:
fname=[]
directory = '/Users/Singsing/Documents/IMDB_Project/Raw_Data/Raw_Data/Actor_mage Data/no_face_crop/'
for subdir, dirs, files in os.walk(directory):
    for file in files:
        filename = file
        fname.append(filename)
fname.pop(0)
fname.pop(0)
print fname     

['1-1000 (106).jpeg', '1-1000 (120).jpeg', '1-1000 (122).jpeg', '1-1000 (127).jpeg', '1-1000 (13).jpeg', '1-1000 (131).jpeg', '1-1000 (135).jpeg', '1-1000 (136).jpeg', '1-1000 (141).jpeg', '1-1000 (146).jpeg', '1-1000 (152).jpeg', '1-1000 (157).jpeg', '1-1000 (165).jpeg', '1-1000 (169).jpeg', '1-1000 (176).jpeg', '1-1000 (177).jpeg', '1-1000 (190).jpeg', '1-1000 (192).jpeg', '1-1000 (203).jpeg', '1-1000 (215).jpeg', '1-1000 (217).jpeg', '1-1000 (218).jpeg', '1-1000 (222).jpeg', '1-1000 (224).jpeg', '1-1000 (226).jpeg', '1-1000 (238).jpeg', '1-1000 (244).jpeg', '1-1000 (247).jpeg', '1-1000 (249).jpeg', '1-1000 (25).jpeg', '1-1000 (253).jpeg', '1-1000 (256).jpeg', '1-1000 (261).jpeg', '1-1000 (262).jpeg', '1-1000 (276).jpeg', '1-1000 (280).jpeg', '1-1000 (282).jpeg', '1-1000 (283).jpeg', '1-1000 (286).jpeg', '1-1000 (289).jpeg', '1-1000 (297).jpeg', '1-1000 (299).jpeg', '1-1000 (300).jpeg', '1-1000 (301).jpeg', '1-1000 (332).jpeg', '1-1000 (334).jpeg', '1-1000 (340).jpeg', '1-1000 (341).

In [ ]:
for f in fname:
    faceCrop(directory+f)

No faces found: /Users/Singsing/Documents/IMDB_Project/Raw_Data/Raw_Data/Actor_mage Data/no_face_crop/1-1000 (106).jpeg
No faces found: /Users/Singsing/Documents/IMDB_Project/Raw_Data/Raw_Data/Actor_mage Data/no_face_crop/1-1000 (120).jpeg
No faces found: /Users/Singsing/Documents/IMDB_Project/Raw_Data/Raw_Data/Actor_mage Data/no_face_crop/1-1000 (122).jpeg
No faces found: /Users/Singsing/Documents/IMDB_Project/Raw_Data/Raw_Data/Actor_mage Data/no_face_crop/1-1000 (127).jpeg
No faces found: /Users/Singsing/Documents/IMDB_Project/Raw_Data/Raw_Data/Actor_mage Data/no_face_crop/1-1000 (13).jpeg
No faces found: /Users/Singsing/Documents/IMDB_Project/Raw_Data/Raw_Data/Actor_mage Data/no_face_crop/1-1000 (131).jpeg
No faces found: /Users/Singsing/Documents/IMDB_Project/Raw_Data/Raw_Data/Actor_mage Data/no_face_crop/1-1000 (135).jpeg
No faces found: /Users/Singsing/Documents/IMDB_Project/Raw_Data/Raw_Data/Actor_mage Data/no_face_crop/1-1000 (146).jpeg
No faces found: /Users/Singsing/Documents